In [1]:
import numpy as np
import sys
from pathlib import Path
import torch
import itertools
import gzip
import argparse
# Local includes
from src.configs.general_opts import path_opts
from src.pdb.input_output import write_precomputed

/home/dima/miniconda3/envs/amber/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
from src.features.data_encoding import numpy_structure
from src.geometry_processing import get_interface_for_pair, get_atom_features, atoms_to_points_normals
from src.geometry_processing import knn_atoms

In [2]:
pid ='6BOY'
cid = 'C'

In [4]:
chain_dir = Path(path_opts["pdb_chain_dir"])
# TODO add check for existing folder. If it exists, skip the computing.
comp_dir = Path(path_opts["precomputed_dir"]) / pid

In [8]:
p_filename = chain_dir / f"{pid}_{cid}.pdb"
protein_encoded = numpy_structure(str(p_filename))
xyz = protein_encoded[-1]  # coordinates

In [14]:
# TODO re-write it for tensorflow
xyz_pt = torch.Tensor(xyz)
b = torch.zeros((xyz_pt.size()[0],), dtype=torch.int8)  # batch numbers, here is only one batch.
P, n, bm = atoms_to_points_normals(xyz_pt, b)

In [18]:
all_features = np.hstack(protein_encoded[:-1])  # all features except coordinates


In [20]:
# protein_encoded

In [39]:
x =  torch.tensor(P)
y = torch.tensor(xyz_pt)
a = torch.tensor(all_features)

/tmp/ipykernel_264684/169000176.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x =  torch.tensor(P)
/tmp/ipykernel_264684/169000176.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(xyz_pt)


In [82]:
a.size()

torch.Size([2107, 106])

In [94]:
idx, dists = knn_atoms(x, y, k=16)  # (num_points, k)
num_points, _ = idx.size()

In [96]:
dists[0]

tensor([ 0.4676,  2.6147,  4.0737,  4.9366,  8.0192, 18.0345, 19.1126, 19.3173,
        30.8753, 31.2411, 34.3915, 42.3332, 43.0744, 45.5213, 46.4158, 47.3844])

In [86]:
idx = idx.view(-1)
dists = 1 / dists.view(-1, 1)
_, num_dims = a.size()

feature = a[idx, :]
feature = torch.cat([feature, dists], dim=1)
feature = feature.view(num_points, 16, num_dims + 1)

In [97]:
mean_features = (feature[:, :, :-1] * feature[:, :, -1][:, :, None]).sum(dim=1)


In [103]:
feature[0][0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 2.1385])

In [104]:
mean_features[10]

tensor([0.9240, 0.0000, 0.0801, 0.0000, 0.0000, 2.5436, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0426, 0.0000, 0.0878, 3.4173, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0783, 0.0801, 0.0862, 0.0000, 0.1440, 0.4673, 0.0000, 0.0000, 0.0000,
        0.0000, 0.1481, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.1118, 0.0000, 0.0000, 0.0000, 0.0426, 2.3892])